### I. 구슬탈출2
#### 1. 내 풀이 (오답)
> blue의 visited 중복문제, 특정 조건시 무한루프 발생

In [ ]:
from collections import deque
import sys
sys.stdin = open('input_13460.txt','r')
sys.setrecursionlimit(1000)

dx = [0,0,1,-1]
dy = [1,-1,0,0]
directions = ["left","right","up","down"]

def go_left(graph, visited, x, y):
    move = 0
    while graph[x][y-1] == '.':
        if not visited[x][y-1]:
            y, move = y-1, move+1
            visited[x][y] = True
        else:
            break
    if graph[x][y-1] == 'O':
        visited[x][y-1] = True
        return x, y-1, move+1
    return x, y, move

def go_right(graph, visited, x, y):
    move = 0
    while graph[x][y+1] == '.':
        if not visited[x][y+1]:
            y, move = y+1, move+1
            visited[x][y] = True
        else:
            break
    if graph[x][y+1] == 'O':
        visited[x][y + 1] = True
        return x, y+1, move+1
    return x, y, move

def go_up(graph, visited, x, y):
    move = 0
    while graph[x-1][y] == '.':
        if not visited[x-1][y]:
            x, move = x+1, move+1
            visited[x][y] = True
        else:
            break
    if graph[x-1][y] == 'O':
        visited[x-1][y] = True
        return x-1, y, move+1
    return x, y, move

def go_down(graph, visited, x, y):
    move = 0
    while graph[x+1][y] == '.':
        if not visited[x+1][y]:
            x, move = x+1, move+1
            visited[x][y] = True
        else:
            break
    if graph[x+1][y] == 'O':
        visited[x + 1][y] = True
        return x+1, y, move+1
    return x, y, move

def BFS(graph,R,B,O):
    queue = deque([(R,0,B,0,0)])
    r_visited = b_visited = [[False]*M for _ in range(N)]
    x1,y1 = R
    x2,y2 = B
    r_visited[x1][y1], b_visited[x2][y2] = True, True
    while queue:
        R, r_move ,B ,b_move,cnt = queue.popleft()
        if B == O and R != O:
            return -1
        if R == O and B != O:
            if cnt <=10:
                return cnt
        r_x, r_y = R
        b_x, b_y = B
        print(r_x, r_y)
        for move in directions:
            if move == "left":
                r_x1, r_y1, r_move = go_left(graph, r_visited, r_x,r_y)
                b_x1, b_y1, b_move = go_left(graph, b_visited, b_x,b_y)
                r, b = (r_x1, r_y1), (b_x1, b_y1)
                queue.append((r,r_move,b,b_move,cnt+1))
            if move == "right":
                r_x1, r_y1, r_move = go_right(graph, r_visited, r_x, r_y)
                b_x1, b_y1, b_move = go_right(graph, b_visited, b_x, b_y)
                r, b = (r_x1, r_y1), (b_x1, b_y1)
                queue.append((r, r_move, b, b_move, cnt + 1))
            if move == "up":
                r_x1, r_y1, r_move = go_up(graph, r_visited, r_x, r_y)
                b_x1, b_y1, b_move = go_up(graph, b_visited, b_x, b_y)
                r, b = (r_x1, r_y1), (b_x1, b_y1)
                queue.append((r, r_move, b, b_move, cnt + 1))
            if move == "down":
                r_x1, r_y1, r_move = go_down(graph, r_visited, r_x, r_y)
                b_x1, b_y1, b_move = go_down(graph, b_visited, b_x, b_y)
                r, b = (r_x1, r_y1), (b_x1, b_y1)
                queue.append((r, r_move, b, b_move, cnt + 1))
    return -1

N, M = map(int,input().split())
graph = [[] for i in range(N)]
R,B,O = (0,0),(0,0),(0,0)
for i in range(N):
    temp = list(input())
    graph[i] = temp
    if 'R' in temp:
        j = temp.index('R')
        R = (i,j)
    if 'B' in temp:
        j = temp.index('B')
        B = (i,j)
    if 'O' in temp:
        j = temp.index('O')
        O = (i,j)
print(BFS(graph,R,B,O))



#### II. 풀이
> blue의 동일 장소 visited를 허용하기 위해 red의 visited를 별도로 두지 않고 4차원으로 해결

In [ ]:
import sys
sys.stdin = open('input_13460.txt','r')

N, M = map(int, input().split())
B = [list(input()) for _ in range(N)]  # Board
dx = [-1, 1, 0, 0]  # x축 움직임
dy = [0, 0, -1, 1]  # y축 움직임
queue = []  # BFS : queue 활용
# Red(rx,ry)와 Blue(bx,by)의 탐사 여부 체크
visited = [[[[False] * M for _ in range(N)] for _ in range(M)] for _ in range(N)]

def pos_init():
    rx, ry, bx, by = 0, 0, 0, 0  # 초기화
    for i in range(N):
        for j in range(M):
            if B[i][j] == 'R':
                rx, ry = i, j
            elif B[i][j] == 'B':
                bx, by = i, j
    # 위치 정보와 depth(breadth 끝나면 +1)
    queue.append((rx, ry, bx, by, 1))
    visited[rx][ry][bx][by] = True

def move(x, y, dx, dy):
    cnt = 0  # 이동 칸 수
    # 다음이 벽이거나 현재가 구멍일 때까지
    while B[x+dx][y+dy] != '#' and B[x][y] != 'O':
        x += dx
        y += dy
        cnt += 1
    return x, y, cnt

def solve():
    pos_init()  # 시작 조건
    while queue:  # BFS : queue 기준
        rx, ry, bx, by, depth = queue.pop(0)
        if depth > 10:  # 실패 조건
            break
        for i in range(4):  # 4방향 이동 시도
            nrx, nry, rcnt = move(rx, ry, dx[i], dy[i])  # Red
            nbx, nby, bcnt = move(bx, by, dx[i], dy[i])  # Blue
            if B[nbx][nby] != 'O':  # 실패 조건이 아니면
                if B[nrx][nry] == 'O':  # 성공 조건
                    print(depth)
                    return
                if nrx == nbx and nry == nby:  # 겹쳤을 때
                    if rcnt > bcnt:  # 이동거리가 많은 것을
                        nrx -= dx[i]  # 한 칸 뒤로
                        nry -= dy[i]
                    else:
                        nbx -= dx[i]
                        nby -= dy[i]
                # breadth 탐색 후, 탐사 여부 체크
                if not visited[nrx][nry][nbx][nby]:
                    visited[nrx][nry][nbx][nby] = True
                    # 다음 depth의 breadth 탐색 위한 queue
                    queue.append((nrx, nry, nbx, nby, depth+1))
    print(-1)  # 실패 시

solve()

### II. B12100 2048(Easy)
#### 1. 내 풀이 (오답)
> [16,16,16,16] -> [32,32] 리턴

In [ ]:
from collections import deque
# import sys
# sys.stdin = open('input_12100.txt','r')

def go_left(board):
    n = len(board)
    temp, tmp = [], [[0]*n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            tmp[i][j] = board[i][j]
    for b in tmp:
        n, t, z = len(b), [], []
        while b:
            data = b.pop(0)
            if data != 0:
                t.append(data)
        k = 1
        for i in range(len(t)-1):
            if t[i] == t[i+1]:
                k += 1
        while t:
            d = t.pop(0)
            if not z:
                z.append(d)
            else:
                if z[-1] == d:
                    d2 = z.pop()
                    z.append(d2+d)
                else:
                    z.append(d)
        while z:
            d = z.pop(0)
            t.append(d)

        for _ in range(n-len(t)):
            t.insert(n-1,0)
        temp.append(t)
    return temp

def go_right(board):
    n = len(board)
    temp, tmp = [], [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            tmp[i][j] = board[i][j]
    for b in tmp:
        n, t, z = len(b), [], []
        while b:
            data = b.pop()
            if data != 0:
                t.append(data)
        while t:
            d = t.pop(0)
            if not z:
                z.append(d)
            else:
                if z[-1] == d:
                    d2 = z.pop()
                    z.append(d2+d)
                else:
                    z.append(d)
        while z:
            d = z.pop()
            t.append(d)

        for _ in range(n-len(t)):
            t.insert(0,0)
        temp.append(t)
    return temp

def go_up(board):
    n = len(board)
    temp = [[0]*n for _ in range(n)]
    temp2 = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            temp[j][i] = board[i][j]
    temp = go_left(temp)
    for j in range(n):
        for i in range(n):
            temp2[i][j] = temp[j][i]
    return temp2

def go_down(board):
    n = len(board)
    temp = [[0]*n for _ in range(n)]
    temp2 = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            temp[j][i] = board[i][j]
    temp = go_right(temp)
    for j in range(n):
        for i in range(n):
            temp2[i][j] = temp[j][i]
    return temp2

def find_max(board):
    MAX = 0
    for b in board:
        MAX = max(MAX,max(b))
    return MAX

def solve(Board):
    queue = deque([(Board,0)])
    result = []
    directions = ["left","right","up","down"]
    while queue:
        board, cnt = queue.popleft()
        if cnt == 5:
            result.append(find_max(board))
        if cnt < 5:
            for move in directions:
                if move == "left":
                    temp_a = go_left(board)
                    queue.append((temp_a,cnt+1))
                if move == "right":
                    temp_b = go_right(board)
                    queue.append((temp_b,cnt+1))
                if move == "up":
                    temp_c = go_up(board)
                    queue.append((temp_c,cnt+1))
                if move == "down":
                    temp_d = go_left(board)
                    queue.append((temp_d,cnt+1))
    return max(result)
N = int(input())
Board = []
for _ in range(N):
    Board.append(list(map(int, input().split())))
print(solve(Board))